In [1]:
import os
import pandas as pd 
import numpy as np
import os, shutil, glob
from random import randint
import re, math
import datetime
import gc
from pathlib import Path

os.getcwd()


'/scratch2-shared/david/colopaint3D/notebooks'

### Read data

In [2]:
pd.set_option("max_colwidth", 2000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)


# BaseDir = '/share/data/analyses/christa/colopaint3D/pilots1-10/results_spheroid'
BaseImgDir = '/share/mikro2/nikon/spheroid-test/pilot12-HCT116-day2/'
BaseDir = '/share/data/cellprofiler/automation/results/pilot12-HCT116-day2/3797/5112'
sorted(os.listdir(BaseDir))

['ICF_CONC.npy',
 'ICF_HOECHST.npy',
 'ICF_MITO.npy',
 'ICF_PHAandWGA.npy',
 'ICF_SYTO.npy',
 'featICF_Image.parquet',
 'featICF_LargeBrightArtefacts_enlarge.parquet',
 'featICF_LargeBrightArtefacts_identify.parquet',
 'featICF_cells.parquet',
 'featICF_cytoplasm.parquet',
 'featICF_nuclei.parquet',
 'featICF_precells.parquet',
 'featICF_prenuclei.parquet',
 'icf_images_pilot12-HCT116-day2.parquet',
 'results']

### Input directories

In [3]:
OutputDir = 'output/1_FeaturesImages'
if not os.path.exists(OutputDir): 
    os.makedirs(OutputDir)
NameContains = ''
InputFolders = pd.read_csv('./settings/filemap.csv')
print(InputFolders )
            


       feature_names
0     featICF_nuclei
1      featICF_cells
2  featICF_cytoplasm


In [4]:
now = datetime.datetime.now()
print ('Current date and time : ')
print (now.strftime('%Y-%m-%d %H:%M:%S'))

Current date and time : 
2023-08-29 14:36:39


In [5]:
# ObjectList = ['featICF_nuclei', 'featICF_cells', 'featICF_cytoplasm']
# cyto = pd.read_parquet(BaseDir+f'/{ObjectList[0]}.parquet').add_prefix('Nuclei_').reset_index()
# ncols = cyto.columns
# for ncol in ncols:
#     print(ncol)

In [6]:
# medianImageAllPlates = pd.DataFrame()
# meanImageAllPlates = pd.DataFrame()
# wellmismatch = 0

# DataFrameDictionary = {}s
ObjectList = ['featICF_nuclei', 'featICF_cells', 'featICF_cytoplasm']
nuclei = pd.read_parquet(BaseDir+f'/{ObjectList[0]}.parquet').add_prefix('Nuclei_').reset_index()
cytoplasm = pd.read_parquet(BaseDir+f'/{ObjectList[1]}.parquet').add_prefix('Cytoplasm_').reset_index()
cells = pd.read_parquet(BaseDir+f'/{ObjectList[2]}.parquet').add_prefix('Cells_').reset_index()

print(nuclei.shape)
print(cytoplasm.shape)
print(cells.shape)

# step 1: Take the mean values of multiple nuclei belonging to one cell
nuclei = nuclei.groupby(["Nuclei_Metadata_Barcode","Nuclei_Metadata_Well","Nuclei_Parent_cells", 'Nuclei_Metadata_Site'], as_index=False).mean(numeric_only=True)
# nuclei = nuclei.groupby(["Nuclei_Metadata_Barcode","Nuclei_Metadata_Well","Nuclei_Parent_cells", 'Nuclei_Metadata_Site'], as_index=False).mean(numeric_only=True)
# step 2: merge nuclei and cytoplasm objects
df = pd.merge(nuclei, cytoplasm, how='left', left_on=['Nuclei_Metadata_Well', 'Nuclei_Metadata_Site',"Nuclei_Parent_cells", 'Nuclei_Metadata_Barcode'],
                right_on = ['Cytoplasm_Metadata_Well','Cytoplasm_Metadata_Site', "Cytoplasm_ObjectNumber", 'Cytoplasm_Metadata_Barcode'])
# step 3: join cells objects
df = pd.merge(df, cells, how='left', left_on=['Cytoplasm_Metadata_Well','Cytoplasm_Metadata_Site',"Cytoplasm_ObjectNumber", 'Cytoplasm_Metadata_Barcode'],
                right_on = ['Cells_Metadata_Well','Cells_Metadata_Site',"Cells_ObjectNumber", 'Cells_Metadata_Barcode'])


# # DataFrameDictionary = {}s
# ObjectList = ['featICF_cytoplasm', 'featICF_nuclei', 'featICF_cells']
# cyto = pd.read_parquet(BaseDir+f'/{ObjectList[0]}.parquet').add_prefix('Cytoplasm_').reset_index()
# nuclei = pd.read_parquet(BaseDir+f'/{ObjectList[1]}.parquet').add_prefix('Nuclei_').reset_index()
# cells = pd.read_parquet(BaseDir+f'/{ObjectList[2]}.parquet').add_prefix('Cells_').reset_index()

# print(cyto.shape)
# print(nuclei.shape)
# print(cells.shape)

# # step 1: Take the mean values of multiple nuclei belonging to one cell
# cyto = cyto.groupby(["Cytoplasm_Metadata_Barcode","Cytoplasm_Metadata_Well","Cytoplasm_Parent_cells", 'Cytoplasm_Metadata_Site'], as_index=False).mean(numeric_only=True)
# # step 2: merge nuclei and cytoplasm objects
# df = pd.merge(cyto, nuclei, how='left', left_on=['Cytoplasm_Metadata_Well', 'Cytoplasm_Metadata_Site',"Cytoplasm_Parent_cells", 'Cytoplasm_Metadata_Barcode'],
#                 right_on = ['Nuclei_Metadata_Well','Nuclei_Metadata_Site', "Nuclei_ObjectNumber", 'Nuclei_Metadata_Barcode'])
# # step 3: join cells objects
# df = pd.merge(df, cells, how='left', left_on=['Nuclei_Metadata_Well','Nuclei_Metadata_Site',"Nuclei_ObjectNumber", 'Nuclei_Metadata_Barcode'],
#                 right_on = ['Cells_Metadata_Well','Cells_Metadata_Site',"Cells_ObjectNumber", 'Cells_Metadata_Barcode'])

print(df.shape)
#------------------- format feature names to BROAD names --------------------------------#
first_column = df.pop('Cytoplasm_Metadata_Barcode')
second_column = df.pop('Cytoplasm_Metadata_Well')
third_column = df.pop('Cytoplasm_Metadata_Site')

df.insert(0, 'Metadata_Barcode', first_column)
df.insert(1, 'Metadata_Well', second_column)
df.insert(2, 'Metadata_Site', third_column)
nameargs = ['Nuclei', 'Cytoplasm', 'Cells']
for name in nameargs:
    df = df.loc[:,~df.columns.str.contains(name+'_Metadata_', case=True)]



# ------------------ add image indentifyer for metdata and QC -----------------------------#
df = df.loc[:,~df.columns.str.contains('Unnamed|index|Cytoplasm_Meta|Cells_Meta|Nuclei_Meta|FileName|PathName|_ImageNumber')]

NrOfObjects = df.shape[0]
print(df['Metadata_Well'])
# Wells = sorted(set(df['Metadata_Well']))
Wells = sorted(list(set(df['Metadata_Well'])))
NrOfWells = len(Wells)
Rows = sorted(list(set([w[0] for w in Wells])))
print(*Rows)
NrOfRows = len(Rows)
Columns = sorted(list(set([w[1:] for w in Wells])))
NrOfColumns = len(Columns)
print(*Columns)
print('Plate complete: {}'.format(NrOfRows*NrOfColumns==NrOfWells))

AllWells =[]
for R in Rows:
    for C in Columns:
        RC = R + C
        # print(RC)
        AllWells += [RC]
# print(len(AllWells))
print('Number of wells: {} (full plate is {})'.format(NrOfWells, len(AllWells)))
# del nuclei
# del cytoplasm
# del cells
dfdb = df.head()
numeric_df = df.select_dtypes(include=[np.number])

### Clipping Outliers, NaNs, and Infs
medians = numeric_df.median()
psu = numeric_df.apply(lambda x: np.nanpercentile(x[x != np.inf], 99), axis=0)
psl = numeric_df.apply(lambda x: np.nanpercentile(x[x != np.inf], 1), axis=0)
numeric_df.fillna(medians, inplace=True)
numeric_df.replace(np.inf, psu, inplace=True)
numeric_df.clip(upper=psu, lower=psl, axis=1, inplace=True)
df[numeric_df.columns] = numeric_df
df = df.copy()
# print(df.)

### Adding Compound Metadata
target = pd.DataFrame()
metaEx = pd.read_csv('/home/jovyan/scratch2-shared/david/colopaint3D/notebooks/settings/pilot12-v2-Adjusted_5-metadata.csv', sep=',')
target['Metadata_Cmpd'] = df['Metadata_Well'].apply(lambda x: metaEx.loc[metaEx['well_id'] == x, 'cmpdname'].values[0])
df.insert(3, 'Metadata_Cmpd', target['Metadata_Cmpd'])
df = df.copy()

### Standardizing to DMSO within Plate.
df_dmso = df[~(df['Metadata_Cmpd'] == 'dmso')]
df_dmso = df_dmso.select_dtypes(include=np.number)
df_dmso = df_dmso.loc[:,~df_dmso.columns.str.contains('Metadata_', case=True)]
std_values = df_dmso.std(skipna=True)
std_values[std_values == 0] = 1
mean_values = df_dmso.mean(skipna=True)

df_norm = df.select_dtypes(include=np.number)
df_norm = df_norm.loc[:,~df_norm.columns.str.contains('Metadata_', case=True)]
df_norm = ((df_norm - mean_values) / std_values)
df[df_norm.columns] = df_norm
df = df.copy()

del nuclei
del cytoplasm
del cells

df.to_parquet('{}/spheroidSingleCell.parquet'.format(OutputDir))

### Mean and Median compositions
df['Metadata_ID'] = df['Metadata_Barcode'] + '+' + df['Metadata_Well'] + '+' + df['Metadata_Cmpd']
metadf = pd.DataFrame()
dictMedian = { i : np.nanmedian for i in numeric_df}
mediandf = df.groupby('Metadata_ID').agg(dictMedian).reset_index(drop=False)
metadatacol = mediandf.pop('Metadata_ID')
metadf[['Metadata_Barcode','Metadata_Well','Metadata_Cmpd']] = metadatacol.str.split('+', expand=True)
mediandf.insert(0, 'Metadata_Barcode', metadf['Metadata_Barcode'])
mediandf.insert(1, 'Metadata_Well', metadf['Metadata_Well'])
mediandf.insert(2, 'Metadata_Cmpd', metadf['Metadata_Cmpd'])

dictMean = { i : np.nanmean for i in numeric_df}
meandf = df.groupby('Metadata_ID').agg(dictMean).reset_index(drop=False)
metadatacol = meandf.pop('Metadata_ID')
metadf[['Metadata_Barcode','Metadata_Well','Metadata_Cmpd']] = metadatacol.str.split('+', expand=True)
meandf.insert(0, 'Metadata_Barcode', metadf['Metadata_Barcode'])
meandf.insert(1, 'Metadata_Well', metadf['Metadata_Well'])
meandf.insert(2, 'Metadata_Cmpd', metadf['Metadata_Cmpd'])
del metadatacol
del metadf
gc.collect()

mediandf.to_parquet('{}/spheroidMedian.parquet'.format(OutputDir))
meandf.to_parquet('{}/spheroidMean.parquet'.format(OutputDir))


(82727, 755)
(71692, 725)
(71692, 716)
(71692, 2176)
0        B02
1        B02
2        B02
3        B02
4        B02
        ... 
71687    O23
71688    O23
71689    O23
71690    O23
71691    O23
Name: Metadata_Well, Length: 71692, dtype: object
B C D E F G H I J K L M N O
02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 18 19 20 21 22 23
Plate complete: False
Number of wells: 241 (full plate is 308)


In [7]:
now = datetime.datetime.now()
print ('Current date and time : ')
print (now.strftime('%Y-%m-%d %H:%M:%S'))

Current date and time : 
2023-08-29 14:37:59


In [8]:
df.head()

,Metadata_Barcode,Metadata_Well,Metadata_Site,Metadata_Cmpd,Nuclei_Parent_cells,Nuclei_ObjectNumber,Nuclei_AreaShape_Area,Nuclei_AreaShape_BoundingBoxArea,Nuclei_AreaShape_BoundingBoxMaximum_X,Nuclei_AreaShape_BoundingBoxMaximum_Y,Nuclei_AreaShape_BoundingBoxMinimum_X,Nuclei_AreaShape_BoundingBoxMinimum_Y,Nuclei_AreaShape_Center_X,Nuclei_AreaShape_Center_Y,Nuclei_AreaShape_Compactness,Nuclei_AreaShape_ConvexArea,Nuclei_AreaShape_Eccentricity,Nuclei_AreaShape_EquivalentDiameter,Nuclei_AreaShape_EulerNumber,Nuclei_AreaShape_Extent,Nuclei_AreaShape_FormFactor,Nuclei_AreaShape_MajorAxisLength,Nuclei_AreaShape_MaxFeretDiameter,Nuclei_AreaShape_MaximumRadius,Nuclei_AreaShape_MeanRadius,Nuclei_AreaShape_MedianRadius,Nuclei_AreaShape_MinFeretDiameter,Nuclei_AreaShape_MinorAxisLength,Nuclei_AreaShape_Orientation,Nuclei_AreaShape_Perimeter,Nuclei_AreaShape_Solidity,Nuclei_AreaShape_Zernike_0_0,Nuclei_AreaShape_Zernike_1_1,Nuclei_AreaShape_Zernike_2_0,Nuclei_AreaShape_Zernike_2_2,Nuclei_AreaShape_Zernike_3_1,Nuclei_AreaShape_Zernike_3_3,Nuclei_AreaShape_Zernike_4_0,Nuclei_AreaShape_Zernike_4_2,Nuclei_AreaShape_Zernike_4_4,Nuclei_AreaShape_Zernike_5_1,Nuclei_AreaShape_Zernike_5_3,Nuclei_AreaShape_Zernike_5_5,Nuclei_AreaShape_Zernike_6_0,Nuclei_AreaShape_Zernike_6_2,Nuclei_AreaShape_Zernike_6_4,Nuclei_AreaShape_Zernike_6_6,Nuclei_AreaShape_Zernike_7_1,Nuclei_AreaShape_Zernike_7_3,Nuclei_AreaShape_Zernike_7_5,Nuclei_AreaShape_Zernike_7_7,Nuclei_AreaShape_Zernike_8_0,Nuclei_AreaShape_Zernike_8_2,Nuclei_AreaShape_Zernike_8_4,Nuclei_AreaShape_Zernike_8_6,Nuclei_AreaShape_Zernike_8_8,Nuclei_AreaShape_Zernike_9_1,Nuclei_AreaShape_Zernike_9_3,Nuclei_AreaShape_Zernike_9_5,Nuclei_AreaShape_Zernike_9_7,Nuclei_AreaShape_Zernike_9_9,Nuclei_Children_cytoplasm_Count,Nuclei_Correlation_Correlation_CONC_HOECHST,Nuclei_Correlation_Correlation_CONC_MITO,Nuclei_Correlation_Correlation_CONC_PHAandWGA,Nuclei_Correlation_Correlation_CONC_SYTO,Nuclei_Correlation_Correlation_HOECHST_MITO,Nuclei_Correlation_Correlation_HOECHST_PHAandWGA,Nuclei_Correlation_Correlation_HOECHST_SYTO,Nuclei_Correlation_Correlation_MITO_PHAandWGA,Nuclei_Correlation_Correlation_MITO_SYTO,Nuclei_Correlation_Correlation_PHAandWGA_SYTO,Nuclei_Correlation_Costes_CONC_HOECHST,Nuclei_Correlation_Costes_CONC_MITO,Nuclei_Correlation_Costes_CONC_PHAandWGA,Nuclei_Correlation_Costes_CONC_SYTO,Nuclei_Correlation_Costes_HOECHST_CONC,Nuclei_Correlation_Costes_HOECHST_MITO,Nuclei_Correlation_Costes_HOECHST_PHAandWGA,Nuclei_Correlation_Costes_HOECHST_SYTO,Nuclei_Correlation_Costes_MITO_CONC,Nuclei_Correlation_Costes_MITO_HOECHST,Nuclei_Correlation_Costes_MITO_PHAandWGA,Nuclei_Correlation_Costes_MITO_SYTO,Nuclei_Correlation_Costes_PHAandWGA_CONC,Nuclei_Correlation_Costes_PHAandWGA_HOECHST,Nuclei_Correlation_Costes_PHAandWGA_MITO,Nuclei_Correlation_Costes_PHAandWGA_SYTO,Nuclei_Correlation_Costes_SYTO_CONC,Nuclei_Correlation_Costes_SYTO_HOECHST,Nuclei_Correlation_Costes_SYTO_MITO,Nuclei_Correlation_Costes_SYTO_PHAandWGA,Nuclei_Correlation_K_CONC_HOECHST,Nuclei_Correlation_K_CONC_MITO,Nuclei_Correlation_K_CONC_PHAandWGA,Nuclei_Correlation_K_CONC_SYTO,Nuclei_Correlation_K_HOECHST_CONC,Nuclei_Correlation_K_HOECHST_MITO,Nuclei_Correlation_K_HOECHST_PHAandWGA,Nuclei_Correlation_K_HOECHST_SYTO,Nuclei_Correlation_K_MITO_CONC,Nuclei_Correlation_K_MITO_HOECHST,Nuclei_Correlation_K_MITO_PHAandWGA,Nuclei_Correlation_K_MITO_SYTO,Nuclei_Correlation_K_PHAandWGA_CONC,Nuclei_Correlation_K_PHAandWGA_HOECHST,Nuclei_Correlation_K_PHAandWGA_MITO,Nuclei_Correlation_K_PHAandWGA_SYTO,Nuclei_Correlation_K_SYTO_CONC,Nuclei_Correlation_K_SYTO_HOECHST,Nuclei_Correlation_K_SYTO_MITO,Nuclei_Correlation_K_SYTO_PHAandWGA,Nuclei_Correlation_Manders_CONC_HOECHST,Nuclei_Correlation_Manders_CONC_MITO,Nuclei_Correlation_Manders_CONC_PHAandWGA,Nuclei_Correlation_Manders_CONC_SYTO,Nuclei_Correlation_Manders_HOECHST_CONC,Nuclei_Correlation_Manders_HOECHST_MITO,Nuclei_Correlation_Manders_HOECHST_PHAandWGA,Nuclei_Correlation_Manders_HOECHST_SYTO

In [9]:
meandf.head()

,Metadata_Barcode,Metadata_Well,Metadata_Cmpd,Metadata_Site,Nuclei_Parent_cells,Nuclei_ObjectNumber,Nuclei_AreaShape_Area,Nuclei_AreaShape_BoundingBoxArea,Nuclei_AreaShape_BoundingBoxMaximum_X,Nuclei_AreaShape_BoundingBoxMaximum_Y,Nuclei_AreaShape_BoundingBoxMinimum_X,Nuclei_AreaShape_BoundingBoxMinimum_Y,Nuclei_AreaShape_Center_X,Nuclei_AreaShape_Center_Y,Nuclei_AreaShape_Compactness,Nuclei_AreaShape_ConvexArea,Nuclei_AreaShape_Eccentricity,Nuclei_AreaShape_EquivalentDiameter,Nuclei_AreaShape_EulerNumber,Nuclei_AreaShape_Extent,Nuclei_AreaShape_FormFactor,Nuclei_AreaShape_MajorAxisLength,Nuclei_AreaShape_MaxFeretDiameter,Nuclei_AreaShape_MaximumRadius,Nuclei_AreaShape_MeanRadius,Nuclei_AreaShape_MedianRadius,Nuclei_AreaShape_MinFeretDiameter,Nuclei_AreaShape_MinorAxisLength,Nuclei_AreaShape_Orientation,Nuclei_AreaShape_Perimeter,Nuclei_AreaShape_Solidity,Nuclei_AreaShape_Zernike_0_0,Nuclei_AreaShape_Zernike_1_1,Nuclei_AreaShape_Zernike_2_0,Nuclei_AreaShape_Zernike_2_2,Nuclei_AreaShape_Zernike_3_1,Nuclei_AreaShape_Zernike_3_3,Nuclei_AreaShape_Zernike_4_0,Nuclei_AreaShape_Zernike_4_2,Nuclei_AreaShape_Zernike_4_4,Nuclei_AreaShape_Zernike_5_1,Nuclei_AreaShape_Zernike_5_3,Nuclei_AreaShape_Zernike_5_5,Nuclei_AreaShape_Zernike_6_0,Nuclei_AreaShape_Zernike_6_2,Nuclei_AreaShape_Zernike_6_4,Nuclei_AreaShape_Zernike_6_6,Nuclei_AreaShape_Zernike_7_1,Nuclei_AreaShape_Zernike_7_3,Nuclei_AreaShape_Zernike_7_5,Nuclei_AreaShape_Zernike_7_7,Nuclei_AreaShape_Zernike_8_0,Nuclei_AreaShape_Zernike_8_2,Nuclei_AreaShape_Zernike_8_4,Nuclei_AreaShape_Zernike_8_6,Nuclei_AreaShape_Zernike_8_8,Nuclei_AreaShape_Zernike_9_1,Nuclei_AreaShape_Zernike_9_3,Nuclei_AreaShape_Zernike_9_5,Nuclei_AreaShape_Zernike_9_7,Nuclei_AreaShape_Zernike_9_9,Nuclei_Children_cytoplasm_Count,Nuclei_Correlation_Correlation_CONC_HOECHST,Nuclei_Correlation_Correlation_CONC_MITO,Nuclei_Correlation_Correlation_CONC_PHAandWGA,Nuclei_Correlation_Correlation_CONC_SYTO,Nuclei_Correlation_Correlation_HOECHST_MITO,Nuclei_Correlation_Correlation_HOECHST_PHAandWGA,Nuclei_Correlation_Correlation_HOECHST_SYTO,Nuclei_Correlation_Correlation_MITO_PHAandWGA,Nuclei_Correlation_Correlation_MITO_SYTO,Nuclei_Correlation_Correlation_PHAandWGA_SYTO,Nuclei_Correlation_Costes_CONC_HOECHST,Nuclei_Correlation_Costes_CONC_MITO,Nuclei_Correlation_Costes_CONC_PHAandWGA,Nuclei_Correlation_Costes_CONC_SYTO,Nuclei_Correlation_Costes_HOECHST_CONC,Nuclei_Correlation_Costes_HOECHST_MITO,Nuclei_Correlation_Costes_HOECHST_PHAandWGA,Nuclei_Correlation_Costes_HOECHST_SYTO,Nuclei_Correlation_Costes_MITO_CONC,Nuclei_Correlation_Costes_MITO_HOECHST,Nuclei_Correlation_Costes_MITO_PHAandWGA,Nuclei_Correlation_Costes_MITO_SYTO,Nuclei_Correlation_Costes_PHAandWGA_CONC,Nuclei_Correlation_Costes_PHAandWGA_HOECHST,Nuclei_Correlation_Costes_PHAandWGA_MITO,Nuclei_Correlation_Costes_PHAandWGA_SYTO,Nuclei_Correlation_Costes_SYTO_CONC,Nuclei_Correlation_Costes_SYTO_HOECHST,Nuclei_Correlation_Costes_SYTO_MITO,Nuclei_Correlation_Costes_SYTO_PHAandWGA,Nuclei_Correlation_K_CONC_HOECHST,Nuclei_Correlation_K_CONC_MITO,Nuclei_Correlation_K_CONC_PHAandWGA,Nuclei_Correlation_K_CONC_SYTO,Nuclei_Correlation_K_HOECHST_CONC,Nuclei_Correlation_K_HOECHST_MITO,Nuclei_Correlation_K_HOECHST_PHAandWGA,Nuclei_Correlation_K_HOECHST_SYTO,Nuclei_Correlation_K_MITO_CONC,Nuclei_Correlation_K_MITO_HOECHST,Nuclei_Correlation_K_MITO_PHAandWGA,Nuclei_Correlation_K_MITO_SYTO,Nuclei_Correlation_K_PHAandWGA_CONC,Nuclei_Correlation_K_PHAandWGA_HOECHST,Nuclei_Correlation_K_PHAandWGA_MITO,Nuclei_Correlation_K_PHAandWGA_SYTO,Nuclei_Correlation_K_SYTO_CONC,Nuclei_Correlation_K_SYTO_HOECHST,Nuclei_Correlation_K_SYTO_MITO,Nuclei_Correlation_K_SYTO_PHAandWGA,Nuclei_Correlation_Manders_CONC_HOECHST,Nuclei_Correlation_Manders_CONC_MITO,Nuclei_Correlation_Manders_CONC_PHAandWGA,Nuclei_Correlation_Manders_CONC_SYTO,Nuclei_Correlation_Manders_HOECHST_CONC,Nuclei_Correlation_Manders_HOECHST_MITO,Nuclei_Correlation_Manders_HOECHST_PHAandWGA,Nuclei_Correlation_Manders_HOECHST_SYTO

In [10]:
toNpy = df.loc[:,~df.columns.str.contains('Metadata_|onehot')]
toNpy.reset_index(inplace=True, drop=True)
toNpy = toNpy.copy()
dataNpy = toNpy.to_numpy()
# colnames = toNpy.columns

In [11]:
np.isnan(dataNpy).any()

False